In [ ]:
# If needed, install scikit-image (this line can be removed if already installed)
!pip install scikit-image

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import convolve2d

def simulate_capture(scene, masks, noise_std=0.01):
    """
    Simulate the sensor measurements by convolving the scene with each mask pattern
    and adding Gaussian noise.
    
    Parameters:
        scene (2D array): The original scene image.
        masks (list of 2D arrays): List of mask patterns.
        noise_std (float): Standard deviation of the additive Gaussian noise.
    
    Returns:
        measurements (list of 2D arrays): Simulated sensor measurements.
    """
    measurements = []
    for mask in masks:
        # 'wrap' boundary condition approximates a periodic assumption
        measurement = convolve2d(scene, mask, mode='same', boundary='wrap')
        # Add Gaussian noise to simulate sensor imperfections
        noise = np.random.randn(*measurement.shape) * noise_std
        measurement_noisy = measurement + noise
        measurements.append(measurement_noisy)
    return measurements

def generate_random_mask(shape, density=0.5):
    """
    Generate a random binary mask.
    
    Parameters:
        shape (tuple): The (height, width) of the mask.
        density (float): Probability that a pixel in the mask is 1.
    
    Returns:
        mask (2D array): A random binary mask.
    """
    return (np.random.rand(*shape) < density).astype(np.float32)

def main():
    # Load a synthetic scene. Try to use the standard 'camera' image from scikit-image.
    try:
        from skimage.data import camera
        scene = camera().astype(np.float32) / 255.0
    except ImportError:
        # Fall back to a random scene if scikit-image is not available
        scene = np.random.rand(256, 256)

    num_masks = 5  # Number of different mask patterns
    mask_shape = scene.shape  # For simplicity, use the same shape as the scene

    # Generate a list of random mask patterns
    masks = [generate_random_mask(mask_shape, density=0.5) for _ in range(num_masks)]

    # Simulate sensor measurements for each mask
    measurements = simulate_capture(scene, masks, noise_std=0.01)

    # Create a plot with the original scene, each mask, and its corresponding measurement
    fig, axes = plt.subplots(num_masks + 1, 2, figsize=(10, 2 * (num_masks + 1)))

    # Display the original scene
    axes[0, 0].imshow(scene, cmap='gray')
    axes[0, 0].set_title('Original Scene')
    axes[0, 0].axis('off')
    axes[0, 1].axis('off')  # Blank for alignment

    # Display each mask and its corresponding measurement
    for i in range(num_masks):
        axes[i+1, 0].imshow(masks[i], cmap='gray')
        axes[i+1, 0].set_title(f'Mask {i+1}')
        axes[i+1, 0].axis('off')

        axes[i+1, 1].imshow(measurements[i], cmap='gray')
        axes[i+1, 1].set_title(f'Measurement {i+1}')
        axes[i+1, 1].axis('off')

    plt.tight_layout()
    plt.show()

# Run the simulation
main()